### Install the required packages

In [ ]:
%pip install python-dotenv
%pip install requests
%pip install azure-storage-blob
%pip install tabulate

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
local_env = secrets.get('LOCAL_URL') or secrets.get("APIM_NAME")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

ragHelper = Helper(local_env, "rag/kpmg", api_key, region_override)
deploymentsHelper = Helper(environment, "genai/azure/deployments", api_key, region_override)

### Create Ingestion for Retreiveal testing

In [ ]:
import datetime

catalogName = f'Regression Catalog: {datetime.datetime.now().strftime("%Y-%m-%dT%H:%M:%S")}'

ragHelper.log("Catalog Name: " + catalogName + '\n')

response = ragHelper.post("/catalog", {
    "displayName": catalogName
})

assert response.status_code == 200

catalogId = response.json["catalogId"]

ragHelper.log("Created Catalog Id: " + catalogId)

from azure.storage.blob import BlobServiceClient, generate_container_sas, ContainerSasPermissions
from datetime import datetime, timedelta, timezone

source_container_name = 'reg-source'
directory_name = 'sourceData'

blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)
account_name = blob_service_client.account_name

source_sas_token = generate_container_sas(
    account_name=account_name,
    container_name=source_container_name,
    account_key=blob_service_client.credential.account_key,
    permission=ContainerSasPermissions(read=True, list=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=24)
)

source_sas_uri = f"https://{account_name}.blob.core.windows.net/{source_container_name}/{directory_name}?{source_sas_token}"

ragHelper.log("Source SAS URI: " + source_sas_uri)

catalogSourceName = f'Regression Catalog Source'

response = ragHelper.post(f"/catalog/{catalogId}/source/blob", {
    "displayName": catalogSourceName,
    "uri": source_sas_uri
})

assert response.status_code == 200

sourceId = response.json["sourceId"]

ragHelper.log("Created Catalog Source Id: " + sourceId)

response = ragHelper.post(f"/catalog/{catalogId}/ingest", None)

assert response.status_code == 202
assert 'ingestionJobId' in response.json

ingestionJobId = response.json["ingestionJobId"]

ragHelper.log("Created Ingestion Job Id: " + ingestionJobId)

#### Attempt with incorrect count

In [ ]:
from IPython.display import display, Markdown

response = ragHelper.post(f"/catalog/{catalogId}/search", {
    'count': -1,
    'search': 'Who is the lead for Azure Hybrid User Benefit best practices?'
})

assert response.status_code == 400
assert response.json["message"] == "The Count value must be greater than 0."

### Attempt with missing count

In [ ]:
from IPython.display import display, Markdown

response = ragHelper.post(f"/catalog/{catalogId}/search", {
    'search': 'Who is the lead for Azure Hybrid User Benefit best practices?'
})

assert response.status_code == 400
assert response.json["message"] == "The Count value must be greater than 0."

### Attempt with wrong Catalog id

In [ ]:
import uuid
from IPython.display import display, Markdown

catalogId = str(uuid.uuid4())
ragHelper.log("Created  Invalid catalogId " + catalogId)
response = ragHelper.post(f"/catalog/{catalogId}/search", {
    'count': 1,
    'search': 'Who is the lead for Azure Hybrid User Benefit best practices?'
})
# commenting out the Assertion as this case is throwing 500 internal Server Error. now considering for fix at this stage (03/25/2025)
#assert response.status_code == 404
#assert response.json["message"] == "The catalog was either not found or you are not the creator."

### Attempt with Empty Search text

In [ ]:
import uuid

catalogId = str(uuid.uuid4())

response = ragHelper.post(f"/catalog/{catalogId}/search", {
    'count': 1,
    'search': ''
})

assert response.status_code == 400
assert response.json["message"] == "The Search text is required."